In [1]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd
import random
from matplotlib.patches import Ellipse
import ligo.skymap.plot
from scipy.optimize import minimize
import numdifftools as ndt
from scipy.interpolate import griddata
from matplotlib.tri import Triangulation
from matplotlib.colors import LogNorm
from matplotlib.pyplot import *

In [2]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/'
scratch_dir = '/pscratch/sd/j/jjpim/'
v_dir = '/pscratch/sd/j/jjpim/massmag/'

# load updated master table
tf_mastertable = Table.read(data_dir + 'tf_mastertable_massmag_stellar.fits') 

#load SGA table
SGA = Table.read('/global/cfs/cdirs/cosmo/data/sga/2020/SGA-2020.fits', 'ELLIPSE')

#load H1 index
SGA_H1 = Table.read('/pscratch/sd/k/kadglass/data/SGA-2020_ALFALFA.fits')
SGA_H1['ALFALFA_index'].name = 'ALFALFA_AGCNr'


alfalfa = Table.read('/pscratch/sd/k/kadglass/data/a100.code12.table2.190808.txt', 
                     format='ascii.csv',
                     include_names=['AGCNr', 'logMH', 'siglogMH', 'W50', 'sigW', 'W20'])

SGA_H1[:5]

SGA_ID,SGA_GALAXY,GALAXY,PGC,RA_LEDA,DEC_LEDA,MORPHTYPE,PA_LEDA,D25_LEDA,BA_LEDA,Z_LEDA,SB_D25_LEDA,MAG_LEDA,BYHAND,REF,GROUP_ID,GROUP_NAME,GROUP_MULT,GROUP_PRIMARY,GROUP_RA,GROUP_DEC,GROUP_DIAMETER,BRICKNAME,RA,DEC,D26,D26_REF,PA,BA,RA_MOMENT,DEC_MOMENT,SMA_MOMENT,G_SMA50,R_SMA50,Z_SMA50,SMA_SB22,SMA_SB22.5,SMA_SB23,SMA_SB23.5,SMA_SB24,SMA_SB24.5,SMA_SB25,SMA_SB25.5,SMA_SB26,G_MAG_SB22,R_MAG_SB22,Z_MAG_SB22,G_MAG_SB22.5,R_MAG_SB22.5,Z_MAG_SB22.5,G_MAG_SB23,R_MAG_SB23,Z_MAG_SB23,G_MAG_SB23.5,R_MAG_SB23.5,Z_MAG_SB23.5,G_MAG_SB24,R_MAG_SB24,Z_MAG_SB24,G_MAG_SB24.5,R_MAG_SB24.5,Z_MAG_SB24.5,G_MAG_SB25,R_MAG_SB25,Z_MAG_SB25,G_MAG_SB25.5,R_MAG_SB25.5,Z_MAG_SB25.5,G_MAG_SB26,R_MAG_SB26,Z_MAG_SB26,SMA_SB22_ERR,SMA_SB22.5_ERR,SMA_SB23_ERR,SMA_SB23.5_ERR,SMA_SB24_ERR,SMA_SB24.5_ERR,SMA_SB25_ERR,SMA_SB25.5_ERR,SMA_SB26_ERR,G_MAG_SB22_ERR,R_MAG_SB22_ERR,Z_MAG_SB22_ERR,G_MAG_SB22.5_ERR,R_MAG_SB22.5_ERR,Z_MAG_SB22.5_ERR,G_MAG_SB23_ERR,R_MAG_SB23_ERR,Z_MAG_SB23_ERR,G_MAG_SB23.5_ERR,R_MAG_SB23.5_ERR,Z_MAG_SB23.5_ERR,G_MAG_SB24_ERR,R_MAG_SB24_ERR,Z_MAG_SB24_ERR,G_MAG_SB24.5_ERR,R_MAG_SB24.5_ERR,Z_MAG_SB24.5_ERR,G_MAG_SB25_ERR,R_MAG_SB25_ERR,Z_MAG_SB25_ERR,G_MAG_SB25.5_ERR,R_MAG_SB25.5_ERR,Z_MAG_SB25.5_ERR,G_MAG_SB26_ERR,R_MAG_SB26_ERR,Z_MAG_SB26_ERR,G_COG_PARAMS_MTOT,G_COG_PARAMS_M0,G_COG_PARAMS_ALPHA1,G_COG_PARAMS_ALPHA2,G_COG_PARAMS_CHI2,R_COG_PARAMS_MTOT,R_COG_PARAMS_M0,R_COG_PARAMS_ALPHA1,R_COG_PARAMS_ALPHA2,R_COG_PARAMS_CHI2,Z_COG_PARAMS_MTOT,Z_COG_PARAMS_M0,Z_COG_PARAMS_ALPHA1,Z_COG_PARAMS_ALPHA2,Z_COG_PARAMS_CHI2,ELLIPSEBIT,ALFALFA_AGCNr
int64,bytes16,bytes29,int64,float64,float64,bytes21,float32,float32,float32,float32,float32,float32,bool,bytes13,int64,bytes35,int16,bool,float64,float64,float32,bytes8,float64,float64,float32,bytes4,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int64
2,SGA-2020 2,PGC1283207,1283207,228.3770865,5.4232017,S?,152.2,0.36307806,0.724436,0.03463229,23.40448,16.976,False,LEDA-20181114,0,PGC1283207,1,True,228.3770865,5.4232017,0.36307806,2283p055,228.3770803831908,5.423191398593787,0.49470574,SB26,158.20142,0.545691,228.37700918822188,5.4232652570544015,10.897086,3.3509698,3.1147978,3.240862,5.902337,6.9126143,7.941369,8.997992,10.073601,11.199986,12.391357,13.561038,14.841172,16.966799,16.108246,15.486356,16.879545,16.024958,15.400715,16.818878,15.967034,15.341793,16.776297,15.925804,15.300776,16.746685,15.897334,15.272053,16.725166,15.876816,15.2521105,16.708357,15.862035,15.237181,16.696539,15.851936,15.226998,16.689613,15.844313,15.21976,0.013392451,0.02354,0.021872982,0.01736985,0.024445537,0.039866067,0.05026544,0.08455789,0.122911856,0.005682776,0.0054258136,0.0049038026,0.005588406,0.005323561,0.0047632363,0.00543534,0.005177031,0.0046343105,0.0053025587,0.005040888,0.0045181247,0.005206092,0.0049438984,0.0044374703,0.0051483097,0.0048758644,0.0043834248,0.0051032505,0.0048264163,0.004344248,0.0050705094,0.004792021,0.004319857,0.005054293,0.004765629,0.0043044444,16.65942,0.34037337,0.2978292,3.0239506,0.07928849,15.820566,0.2640441,0.34559453,3.3033552,0.003811298,15.195567,0.29826432,0.3001073,3.2333765,0.011723555,0,-1
3,SGA-2020 3,PGC1310416,1310416,202.54443750000002,6.9345944,Sc,159.26,0.4017908,0.7816278,0.073888786,23.498482,16.85,False,LEDA-20181114,1,PGC1310416,1,True,202.544437500000

In [3]:
alfalfa[:5]

AGCNr,W50,sigW,W20,logMH,siglogMH
int64,int64,int64,int64,float64,float64
105367,274,39,281,9.87,0.05
333313,313,20,333,10.01,0.05
331060,160,4,184,9.07,0.18
331061,260,45,268,9.29,0.06
104570,245,6,250,9.64,0.06


In [4]:
tf_mastertable[tf_mastertable['SGA_ID']==202236 ]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,Z_CENTER,ROTCURVE_CHI2,VMAX,VMAX_ERR,RTURN,RTURN_ERR,ALPHA,ALPHA_ERR,INCLINATION,VROT_R26,VROT_R26_ERR,M_R26,M_R26_ERR,VROT_3P5R26,VROT_3P5R26_ERR,M_3P5R26,M_3P5R26_ERR,RABS_MAG26,M_STELLAR_CIGALE,M_STELLAR_CIGALE_ERR,M_STELLAR_PROVABGS,M_STELLAR_PROVABGS_ERR
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1070919447478281,218.467126790284,35.6021357274572,10517,sv3,0.689349774676162,0.000105899659336298,4,1.1599750071764,fuji/healpix/sv3/bright/105/10517/redrock-sv3-bright-10517.fits,TFT,202236,218.46712679028371,35.60213572745723,1.0,0.010328013447839866,0.9999999757533122,1.0,0.0,189632.3345066882,31.761552886604264,0.0347949861264779,--,--,--,--,--,--,--,--,--,0.0,--,--,--,0.0,--,--,--,--,--,--,--
1070919447478283,218.473204372212,35.6069798702523,10517,sv3,0.0342643027523248,5.81042710273245e-06,0,521.161364927888,fuji/healpix/sv3/bright/105/10517/redrock-sv3-bright-10517.fits,TFT,202236,218.47320437221234,35.60697987025231,1.0,0.003408244437794044,0.32999999199926,1.0,0.0,-153.74530732177234,1.9748830179274115,0.0347949861264779,57.053585993905905,197.6158181854043,114.27829537570685,0.3001536230733187,0.31094870739361663,2.000207125110637,5.976812825788214,1.4711405903507528,197.59066947535428,873.2614200215331,11.232204063553201,0.992955185455564,197.6137654022168,1241.2374867700123,11.776373629371527,1.1323319805123848,--,9.759396229228562,0.07878821529625392,--,--
39632991256840767,218.476198078587,35.6093656791547,10517,sv3,0.0347951441075021,4.38416406810812e-06,0,3660.84922169894,fuji/healpix/sv3/bright/105/10517/redrock-sv3-bright-10517.fits,TFT,202236,218.47619807858737,35.60936567915472,1.0,0.0,0.0,1.0,0.0,0.0457689882425998,1.6103905603883806,0.0347949861264779,57.053585993905905,197.6158181854043,114.27829537570685,0.3001536230733187,0.31094870739361663,2.000207125110637,5.976812825788214,1.4711405903507528,197.59066947535428,873.2614200215331,11.232204063553201,0.992955185455564,197.6137654022168,1241.2374867700123,11.776373629371527,1.1323319805123848,--,9.759396229228562,0.07878821529625392,--,--
1070919447478282,218.470120134196,35.6045216866714,10517,sv3,0.035417316436115,3.76541421700099e-05,0,9.22480585216545,fuji/healpix/sv3/bright/105/10517/redrock-sv3-bright-10517.fits,TFT,202236,218.47012013419638,35.604521686671355,1.0,0.006919769010041057,0.6699999837535908,1.0,0.0,180.29651835904573,11.32671500875773,0.0347949861264779,57.053585993905905,197.6158181854043,114.27829537570685,0.3001536230733187,0.31094870739361663,2.000207125110637,5.976812825788214,1.4711405903507528,197.59066947535428,873.2614200215331,11.232204063553201,0.992955185455564,197.6137654022168,1241.2374867700123,11.776373629371527,1.1323319805123848,--,9.759396229228562,0.07878821529625392,--,--
1070919447478284,218.47919196352,35.6117514140064,10517,sv3,0.0352543507475819,2.11623843739911e-05,0,39.4392753937282,fuji/healpix/sv3/bright/105/10517/redrock-sv3-bright-10517.fits,TFT,202236,218.47919196352018,35.61175141400644,1.0,0.003408244437787266,0.3299999919986037,1.0,0.0,133.08341335760971,6.412199866829514,0.0347949861264779,57.053585993905905,197.6158181854043,114.27829537570685,0.3001536230733187,0.31094870739361663,2.000207125110637,5.976812825788214,1.4711405903507528,197.59066947535428,873.2614200215331,11.232204063553201,0.992955185455564,197.6137654022168,1241.2374867700123,11.776373629371527,1.1323319805123848,--,9.759396229228562,0.07878821529625392,--,--
1084113587011596,218.47919196352,35.6117514140064,10517,sv3,0.0352300902375281,1.03763672224515e-05,0,221.168140199035,f

In [5]:
sga_ids = [9769, 11977, 12892, 14501, 16383, 19235, 19278, 25532, 30149, 31220, 32484, 36904, 37543, 46327, 58617, 73111, 88963, 91660, 90791, 106536, 
           114405, 114489, 122580, 126630, 125611, 128163, 128411, 130766, 149502, 153504, 175220, 176736, 184044, 186239, 189685, 188525, 190308, 
           192158, 196470, 199038, 202236, 205028, 209429, 212382, 218773, 235556, 254532, 272164, 272562, 276638, 288715, 289743, 291766, 300446, 
           302062, 305831, 309378, 319942, 330678, 349027, 349346, 363344, 373692, 378842, 384420, 389177, 393190, 395070, 398443, 398532, 416531, 
           431242, 430161, 432294, 450689, 450566, 455486, 457147, 458798, 462867, 466872, 467582, 471327, 469868, 471893, 473436, 477805, 481956, 
           485800, 493017, 507499, 526710, 544419, 544810, 544799, 544943, 548724, 548848, 550458, 564440, 569298, 568664, 570086, 582248, 591927, 
           607528, 608015, 629144, 630029, 630390, 635043, 638386, 641578, 643231, 643845, 644940, 647966, 649753, 649960, 651096, 654469, 659599, 
           663463, 671301, 671533, 694525, 696291, 696345, 706579, 720480, 722142, 726854, 728327, 729726, 729754, 736449, 742167, 743030, 744650, 
           752683, 757313, 757166, 758123, 759261, 759003, 766672, 783693, 793633, 804643, 805225, 819754, 820598, 823301, 824225, 826809, 827949, 
           833961, 833981, 843065, 843126, 845052, 887433, 901663, 901980, 908818, 908978, 911046, 914975, 914909, 916161, 920806, 923400, 925923, 
           925354, 928810, 928876, 932984, 947709, 953644, 957521, 957850, 967351, 968864, 971635, 975775, 972260, 977015, 976853, 979168, 978708, 
           989040, 989244, 1001302, 1003319, 1008726, 1014959, 1018187, 1021462, 1026141, 1031898, 1033651, 1033344, 1050173, 1065602, 1073430, 
           1086934, 1087039, 1087374, 1090772, 1091806, 1096564, 1098043, 1103477, 1111824, 1117018, 1124386, 1143335, 1146354, 1161160, 1167691, 
           1171841, 1179377, 1184894, 1184912, 1186034, 1187240, 1188032, 1188617, 1188926, 1193015, 1191665, 1194588, 1194958, 1199020, 1199539, 
           1201319, 1203786, 1204237, 1206707, 1218166, 1219319, 1254154, 1264056, 1264591, 1267967, 1271477, 1271549, 1272700, 1273605, 1278144, 
           1281619, 1282407, 1285273, 1286710, 1289613, 1293967, 1301768, 1304908, 1335916, 1338866, 1345100, 1348956, 1349555, 1354266, 1355575, 
           1361363, 1361399, 1365519, 1365549, 1367137, 1368018, 1368443, 1374964, 1379143, 1405053, 1429680, 1434487]

In [6]:
#new column in tf mastertable for H1 velocities
tf_mastertable['V_HI']=np.nan
tf_mastertable['VERR_HI']=np.nan
tf_mastertable['V20_HI']=np.nan 
#new column in tf mastertable for H1 masses
tf_mastertable['M_HI']=np.nan
tf_mastertable['M_HI_ERR']=np.nan

In [7]:
#make dictionaries
SGA_H1_dict = {}
for i in range(len(SGA_H1)):
    SGA_H1_dict[SGA_H1['SGA_ID'][i]] = i

alfalfa_dict = {}
for i in range(len(alfalfa)):
    alfalfa_dict[alfalfa['AGCNr'][i]] = i

In [8]:
#using to determine # of sga ids in my list that have H1 vals
sga_ids_with_alfalfa = []

for sga_id in sga_ids:
    # Get the ALFALFA_AGCNr value for the current SGA_ID
    alfalfa_agcnr_value = SGA_H1[SGA_H1['SGA_ID'] == sga_id]['ALFALFA_AGCNr']
    
    # Check if the value is greater than 0 and store the SGA_ID if true
    if len(alfalfa_agcnr_value) > 0 and alfalfa_agcnr_value[0] > 0:
        sga_ids_with_alfalfa.append(sga_id)
        
len(sga_ids_with_alfalfa)

50

In [9]:
print(sga_ids_with_alfalfa)

[25532, 46327, 153504, 176736, 188525, 235556, 254532, 291766, 373692, 398443, 450566, 455486, 466872, 473436, 548848, 608015, 630029, 638386, 643231, 654469, 696291, 722142, 726854, 766672, 783693, 804643, 819754, 824225, 887433, 914975, 932984, 947709, 977015, 1018187, 1021462, 1026141, 1033344, 1096564, 1143335, 1146354, 1167691, 1179377, 1194958, 1204237, 1271477, 1272700, 1273605, 1289613, 1304908, 1361363]


In [10]:
#using to determine # of sga ids in tf_mastertable that have H1 vals
all_sga_ids_with_alfalfa = []
all_sga_ids = tf_mastertable['SGA_ID']
for sga_id in all_sga_ids:
    # Get the ALFALFA_AGCNr value for the current SGA_ID
    alfalfa_agcnr_value = SGA_H1[SGA_H1['SGA_ID'] == sga_id]['ALFALFA_AGCNr']
    
    # Check if the value is greater than 0 and store the SGA_ID if true
    if len(alfalfa_agcnr_value) > 0 and alfalfa_agcnr_value[0] > 0:
        all_sga_ids_with_alfalfa.append(sga_id)

In [11]:
unique = np.unique(all_sga_ids_with_alfalfa)
len(unique)

6729

In [12]:
unique_2 = np.unique(all_sga_ids)
len(unique_2)

75171

In [13]:
all_sga_ids = tf_mastertable['SGA_ID']
for sga_id in all_sga_ids: 
    SGA_index = SGA_H1_dict[sga_id] 

    # determine if H1 exists + save velocity into table
    if SGA_H1['ALFALFA_AGCNr'][SGA_index] >= 0:
        alfalfa_index = alfalfa_dict[SGA_H1['ALFALFA_AGCNr'][SGA_index]]
        # update repsective velocity columns in tf_mastertable
        tf_mastertable['V_HI'][tf_mastertable['SGA_ID'] == sga_id] = alfalfa['W50'][alfalfa_index]
        tf_mastertable['VERR_HI'][tf_mastertable['SGA_ID'] == sga_id] = alfalfa['sigW'][alfalfa_index]
        tf_mastertable['V20_HI'][tf_mastertable['SGA_ID'] == sga_id] = alfalfa['W20'][alfalfa_index]
        tf_mastertable['M_HI'][tf_mastertable['SGA_ID'] == sga_id] = alfalfa['logMH'][alfalfa_index]
        tf_mastertable['M_HI_ERR'][tf_mastertable['SGA_ID'] == sga_id] = alfalfa['siglogMH'][alfalfa_index]
        
#test:
tf_mastertable[tf_mastertable['SGA_ID']==819754]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,Z_CENTER,ROTCURVE_CHI2,VMAX,VMAX_ERR,RTURN,RTURN_ERR,ALPHA,ALPHA_ERR,INCLINATION,VROT_R26,VROT_R26_ERR,M_R26,M_R26_ERR,VROT_3P5R26,VROT_3P5R26_ERR,M_3P5R26,M_3P5R26_ERR,RABS_MAG26,M_STELLAR_CIGALE,M_STELLAR_CIGALE_ERR,M_STELLAR_PROVABGS,M_STELLAR_PROVABGS_ERR,V_HI,VERR_HI,V20_HI,M_HI,M_HI_ERR
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1070748764471296,194.109754320792,27.286921909931,10378,sv3,1.36278539636169,0.000105634287518713,4,1.42624063417315,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,TFT,819754,194.1097543207924,27.28692190993096,1.0,0.009381687483714632,0.999999983219526,1.0,0.0,391219.21457314346,31.67441794748501,0.025084336222695403,--,--,--,--,--,--,--,--,--,0.0,--,--,--,0.0,--,--,-19.613118839397558,--,--,--,--,90.0,38.0,115.0,9.24,0.06
1070748764471297,194.112835311759,27.288366767408,10378,sv3,1.27470950412671,8.81266758246102e-05,4,0.76961188018322,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,TFT,819754,194.11283531175852,27.288366767407975,1.0,0.006285730614070416,0.6699999887551223,1.0,0.0,365460.8576354451,26.42697066076073,0.025084336222695403,--,--,--,--,--,--,--,--,--,0.0,--,--,--,0.0,--,--,-19.613118839397558,--,--,--,--,90.0,38.0,115.0,9.24,0.06
1083942904004609,194.112835311759,27.288366767408,10378,sv3,0.0248557654352545,1.38306950988848e-05,0,89.4312909683213,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,TFT,819754,194.11283531175852,27.288366767407975,1.0,0.006285730614070416,0.6699999887551223,1.0,0.0,-66.84698592347671,4.1923353169640984,0.025084336222695403,190.29168313614323,124.35972463172413,--,0.010015034656753154,--,1.0418582190526275,--,0.8384614859685158,124.28768782283315,0.0,10.64567751125536,--,124.34018573918587,0.0,--,--,-19.613118839397558,9.76909358696662,0.09082220976919804,10.38829231262207,0.09357101949269822,90.0,38.0,115.0,9.24,0.06
1070748764471298,194.11600974993,27.2898553378398,10378,sv3,0.024847210269851,6.38884001821522e-06,0,1059.62361641275,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,TFT,819754,194.11600974992967,27.289855337839818,1.0,0.0030959568696263444,0.3299999944624985,1.0,0.0,-69.34899865972052,2.012965431556563,0.025084336222695403,190.29168313614323,124.35972463172413,--,0.010015034656753154,--,1.0418582190526275,--,0.8384614859685158,124.28768782283315,0.0,10.64567751125536,--,124.34018573918587,0.0,--,--,-19.613118839397558,9.76909358696662,0.09082220976919804,10.38829231262207,0.09357101949269822,90.0,38.0,115.0,9.24,0.06
39628422527321535,194.119090903652,27.2913000582538,10378,sv3,0.0250843060436296,2.92035136602822e-06,0,5824.72504919767,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,TFT,819754,194.11909090365222,27.291300058253793,1.0,6.3611093629270335e-15,6.780346571154332e-13,1.0,0.0,-0.008826060407103396,1.0724061680343704,0.025084336222695403,190.29168313614323,124.35972463172413,--,0.010015034656753154,--,1.0418582190526275,--,0.8384614859685158,124.28768782283315,0.0,10.64567751125536,--,124.34018573918587,0.0,--,--,-19.613118839397558,9.76909358696662,0.09082220976919804,10.38829231262207,0.09357101949269822,90.0,38.0,115.0,9.24,0.06
1070748764471299,194.122172137543,27.2927447111501,10378,sv3,0.0254181403646655,2.88112847075796e-06,0,2614.82661251724,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,TFT,819754,194.1221721375434,27.29274471115013,1.0,0.003095956869632472,0.32999999446315165,1.0,0.0,97.62315223798166,1.0628281799075092,0.0250843362226

In [14]:
tf_mastertable[tf_mastertable['SGA_ID']==819754]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VERR,Z_CENTER,ROTCURVE_CHI2,VMAX,VMAX_ERR,RTURN,RTURN_ERR,ALPHA,ALPHA_ERR,INCLINATION,VROT_R26,VROT_R26_ERR,M_R26,M_R26_ERR,VROT_3P5R26,VROT_3P5R26_ERR,M_3P5R26,M_3P5R26_ERR,RABS_MAG26,M_STELLAR_CIGALE,M_STELLAR_CIGALE_ERR,M_STELLAR_PROVABGS,M_STELLAR_PROVABGS_ERR,V_HI,VERR_HI,V20_HI,M_HI,M_HI_ERR
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1070748764471296,194.109754320792,27.286921909931,10378,sv3,1.36278539636169,0.000105634287518713,4,1.42624063417315,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,TFT,819754,194.1097543207924,27.28692190993096,1.0,0.009381687483714632,0.999999983219526,1.0,0.0,391219.21457314346,31.67441794748501,0.025084336222695403,--,--,--,--,--,--,--,--,--,0.0,--,--,--,0.0,--,--,-19.613118839397558,--,--,--,--,90.0,38.0,115.0,9.24,0.06
1070748764471297,194.112835311759,27.288366767408,10378,sv3,1.27470950412671,8.81266758246102e-05,4,0.76961188018322,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,TFT,819754,194.11283531175852,27.288366767407975,1.0,0.006285730614070416,0.6699999887551223,1.0,0.0,365460.8576354451,26.42697066076073,0.025084336222695403,--,--,--,--,--,--,--,--,--,0.0,--,--,--,0.0,--,--,-19.613118839397558,--,--,--,--,90.0,38.0,115.0,9.24,0.06
1083942904004609,194.112835311759,27.288366767408,10378,sv3,0.0248557654352545,1.38306950988848e-05,0,89.4312909683213,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,TFT,819754,194.11283531175852,27.288366767407975,1.0,0.006285730614070416,0.6699999887551223,1.0,0.0,-66.84698592347671,4.1923353169640984,0.025084336222695403,190.29168313614323,124.35972463172413,--,0.010015034656753154,--,1.0418582190526275,--,0.8384614859685158,124.28768782283315,0.0,10.64567751125536,--,124.34018573918587,0.0,--,--,-19.613118839397558,9.76909358696662,0.09082220976919804,10.38829231262207,0.09357101949269822,90.0,38.0,115.0,9.24,0.06
1070748764471298,194.11600974993,27.2898553378398,10378,sv3,0.024847210269851,6.38884001821522e-06,0,1059.62361641275,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,TFT,819754,194.11600974992967,27.289855337839818,1.0,0.0030959568696263444,0.3299999944624985,1.0,0.0,-69.34899865972052,2.012965431556563,0.025084336222695403,190.29168313614323,124.35972463172413,--,0.010015034656753154,--,1.0418582190526275,--,0.8384614859685158,124.28768782283315,0.0,10.64567751125536,--,124.34018573918587,0.0,--,--,-19.613118839397558,9.76909358696662,0.09082220976919804,10.38829231262207,0.09357101949269822,90.0,38.0,115.0,9.24,0.06
39628422527321535,194.119090903652,27.2913000582538,10378,sv3,0.0250843060436296,2.92035136602822e-06,0,5824.72504919767,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,TFT,819754,194.11909090365222,27.291300058253793,1.0,6.3611093629270335e-15,6.780346571154332e-13,1.0,0.0,-0.008826060407103396,1.0724061680343704,0.025084336222695403,190.29168313614323,124.35972463172413,--,0.010015034656753154,--,1.0418582190526275,--,0.8384614859685158,124.28768782283315,0.0,10.64567751125536,--,124.34018573918587,0.0,--,--,-19.613118839397558,9.76909358696662,0.09082220976919804,10.38829231262207,0.09357101949269822,90.0,38.0,115.0,9.24,0.06
1070748764471299,194.122172137543,27.2927447111501,10378,sv3,0.0254181403646655,2.88112847075796e-06,0,2614.82661251724,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,TFT,819754,194.1221721375434,27.29274471115013,1.0,0.003095956869632472,0.32999999446315165,1.0,0.0,97.62315223798166,1.0628281799075092,0.0250843362226

In [15]:
#writing new data into new table
tf_mastertable.write(data_dir + 'tf_mastertable_massmag_stellar_H1.fits', format='fits', overwrite=True)